In [0]:
pip install -q pyyaml h5py  # Required to save models in HDF5 format

In [2]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.2.0-rc3


In [0]:
(trainset, trainlabels), (testset, testlabels) = tf.keras.datasets.mnist.load_data()

trainlabels = trainlabels[:1000]
testlabels = testlabels[:1000]

trainset = trainset[:1000].reshape(-1, 28 * 28) / 255.0
testset = testset[:1000].reshape(-1, 28 * 28) / 255.0

In [5]:
# Define a simple sequential model
def createmodel():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

# Create a basic model instance
mm = createmodel()

# Display the model's architecture
mm.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [6]:
checkpointpath = "training_1/cp.ckpt"
checkpointdir = os.path.dirname(checkpointpath)

# Create a callback that saves the model's weights
cpcallback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpointpath,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
mm.fit(trainset, 
          trainlabels,  
          epochs=10,
          validation_data=(testset,testlabels),
          callbacks=[cpcallback])  # Pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

Epoch 1/10
22/32 [===================>..........] - ETA: 0s - loss: 1.4184 - accuracy: 0.6122
Epoch 00001: saving model to training_1/cp.ckpt
32/32 [==============================] - 1s 16ms/step - loss: 1.1947 - accuracy: 0.6760 - val_loss: 0.7611 - val_accuracy: 0.7560
Epoch 2/10
25/32 [======================>.......] - ETA: 0s - loss: 0.4698 - accuracy: 0.8675
Epoch 00002: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 7ms/step - loss: 0.4486 - accuracy: 0.8770 - val_loss: 0.5249 - val_accuracy: 0.8380
Epoch 3/10
27/32 [========================>.....] - ETA: 0s - loss: 0.2968 - accuracy: 0.9271
Epoch 00003: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 7ms/step - loss: 0.2827 - accuracy: 0.9310 - val_loss: 0.4680 - val_accuracy: 0.8490
Epoch 4/10
24/32 [=====================>........] - ETA: 0s - loss: 0.2051 - accuracy: 0.9531
Epoch 00004: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 

In [7]:
ls {checkpoint_dir}

ls: cannot access '{checkpoint_dir}': No such file or directory


In [9]:
# Create a basic model instance
mm = createmodel()

# Evaluate the model
loss, acc = mm.evaluate(testset,  testlabels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 2.4033 - accuracy: 0.0510
Untrained model, accuracy:  5.10%


In [10]:
# Loads the weights
mm.load_weights(checkpointpath)

# Re-evaluate the model
loss,acc = mm.evaluate(testset,  testlabels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4134 - accuracy: 0.8680
Restored model, accuracy: 86.80%


In [11]:
# Include the epoch in the file name (uses `str.format`)
checkpointpath = "training_2/cp-{epoch:04d}.ckpt"
checkpointdir = os.path.dirname(checkpointpath)

# Create a callback that saves the model's weights every 5 epochs
cpcallback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpointpath, 
    verbose=1, 
    save_weights_only=True,
    period=5)

# Create a new model instance
mm = createmodel()

# Save the weights using the `checkpoint_path` format
mm.save_weights(checkpointpath.format(epoch=0))

# Train the model with the new callback
mm.fit(trainset, 
          trainlabels,
          epochs=50, 
          callbacks=[cpcallback],
          validation_data=(testset,testlabels),
          verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [12]:
ls {checkpoint_dir}

ls: cannot access '{checkpoint_dir}': No such file or directory


In [14]:
latest = tf.train.latest_checkpoint(checkpointdir)
latest

'training_2/cp-0050.ckpt'

In [15]:
# Create a new model instance
mm = createmodel()

# Load the previously saved weights
mm.load_weights(latest)

# Re-evaluate the model
loss, acc = mm.evaluate(testset,  testlabels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4869 - accuracy: 0.8770
Restored model, accuracy: 87.70%


In [16]:
# Save the weights
mm.save_weights('./checkpoints/my_checkpoint')

# Create a new model instance
mm = createmodel()

# Restore the weights
mm.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
loss,acc = mm.evaluate(testset,  testlabels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4869 - accuracy: 0.8770
Restored model, accuracy: 87.70%


In [18]:
# Create and train a new model instance.
mm = createmodel()
mm.fit(trainset, trainlabels, epochs=5)

# Save the entire model as a SavedModel.
!mkdir -p saved_model
mm.save('saved_model/my_model') 

Epoch 1/5
32/32 [==============================] - 0s 4ms/step - loss: 1.1856 - accuracy: 0.6660
Epoch 2/5
32/32 [==============================] - 0s 4ms/step - loss: 0.4243 - accuracy: 0.8750
Epoch 3/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2991 - accuracy: 0.9250
Epoch 4/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2187 - accuracy: 0.9480
Epoch 5/5
32/32 [==============================] - 0s 4ms/step - loss: 0.1484 - accuracy: 0.9670
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [19]:
# my_model directory
!ls saved_model

# Contains an assets folder, saved_model.pb, and variables folder.
!ls saved_model/my_model

my_model
assets	saved_model.pb	variables


In [20]:
newmodel = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
newmodel.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [21]:
# Evaluate the restored model
loss, acc = newmodel.evaluate(testset,  testlabels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

print(newmodel.predict(testset).shape)

32/32 - 0s - loss: 0.4272 - accuracy: 0.8670
Restored model, accuracy: 86.70%
(1000, 10)


In [24]:
# Create and train a new model instance.
model = createmodel()
model.fit(trainset, trainlabels, epochs=5)

# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model should be saved to HDF5.
model.save('my_model.h5') 

Epoch 1/5
32/32 [==============================] - 0s 4ms/step - loss: 1.1271 - accuracy: 0.6670
Epoch 2/5
32/32 [==============================] - 0s 4ms/step - loss: 0.4260 - accuracy: 0.8830
Epoch 3/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2936 - accuracy: 0.9230
Epoch 4/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2198 - accuracy: 0.9420
Epoch 5/5
32/32 [==============================] - 0s 3ms/step - loss: 0.1621 - accuracy: 0.9610


In [25]:
# Recreate the exact same model, including its weights and the optimizer
newmodel = tf.keras.models.load_model('my_model.h5')

# Show the model architecture
newmodel.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [26]:
loss, acc = newmodel.evaluate(testset,  testlabels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

32/32 - 0s - loss: 0.4422 - accuracy: 0.8540
Restored model, accuracy: 85.40%
